In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import math
import random
from sklearn.tree import DecisionTreeClassifier

In [12]:
X= np.array(pd.read_csv('./letter-recognition.data'))
print (X[0])
images=X[:,1:]
label=X[:,0]
print(images)
print(label)


['T' 2 8 3 5 1 8 13 0 6 6 10 8 0 8 0 8]
[[2 8 3 ... 8 0 8]
 [5 12 3 ... 8 4 10]
 [4 11 6 ... 7 3 9]
 ...
 [6 9 6 ... 12 2 4]
 [2 3 4 ... 9 5 8]
 [4 9 6 ... 7 2 8]]
['T' 'I' 'D' ... 'T' 'S' 'A']


In [13]:
#PART A
randarray= np.arange(len(X))
np.random.shuffle(randarray)
breakpt= round(0.7*len(images))
X_train= []
Y_train= []
X_test= []
Y_test= []
for i in range(breakpt):
    X_train.append(images[randarray[i]])
    Y_train.append(label[randarray[i]])
for i in range(breakpt, len(images)):
    X_test.append(images[randarray[i]])
    Y_test.append(label[randarray[i]])
    
print (len(X_train), len(X_test))
X_train= np.array(X_train)
Y_train= np.array(Y_train)
print (Y_train[:10])
X_test= np.array(X_test)
Y_test= np.array(Y_test)
Y_train= [ord(char)-65 for char in Y_train]
Y_test= [ord(char)-65 for char in Y_test]

                 

14000 6000
['H' 'N' 'M' 'Y' 'Z' 'B' 'T' 'R' 'G' 'V']


In [14]:
def getIndex(num, cum):
    index= -1
    for i in range(len(cum)):
        if(num<= cum[i]):
            return i #between 0to149 format
    

In [15]:
def getAcc(yy_train, probsmatrix):
    trainpred= []
    for i in range(len(yy_train)):
        trainpred.append(np.argmax(probsmatrix[i]))
    trainpred= np.array(trainpred)
    #print (np.unique(trainpred))
    return ((yy_train== trainpred).sum())/len(yy_train)

In [16]:
def boosting(X_train, Y_train, X_test, N):
    originaldt= np.zeros(len(X_train))
    originaldt+= 1/len(X_train)
    uniq= np.unique(Y_train)
    probstest= np.zeros([len(X_test), len(uniq)])
    probstrain= np.zeros([len(X_train), len(uniq)])
    print (originaldt)
    alphas= []
    for i in range(N):
        print (str(i) +  " " + "Iteration")
        indices= []
        #labeltemp= np.zeros(len(Y_train))
        cum= np.cumsum(originaldt)
        print (cum)
        #temp= np.zeros([len(X_train), len(X_train[0])])
#         for j in range(len(X_train)):
#             num= random.uniform(0, 1)
#             a= getIndex(num, cum)
#             indices.append(a)
#             labeltemp[j]= Y_train[a]
#             temp[j]= X_train[a]
        
#         originaldt=np.zeros(len(X_train))
#         originaldt+= 1/len(X_train)
        #print (labeltemp.shape, temp.shape)
        clf= DecisionTreeClassifier(random_state= 0, max_depth= 2, max_leaf_nodes= 5)
        #clf= clf.fit(temp, labeltemp)
        clf= clf.fit(X_train, Y_train, sample_weight= originaldt)
        #pred= clf.predict(temp)
        pred= clf.predict(X_train)
        pred= np.array(pred)
        error= (((pred!= Y_train)).sum())/len(pred)
        print (error)
        alpha= abs(1/2*math.log((1-error)/(error)))
        alphas.append(alpha)
        print (alpha)
        for j in range(len(pred)):
            if(pred[j]== Y_train[j]):
                originaldt[j]= originaldt[j]*math.exp(-alpha)
            else:
                originaldt[j]= originaldt[j]*math.exp(alpha)
        
        #now normalize the weights updated
        #print (originaldt)
        originaldt= originaldt/np.sum(originaldt)#normalize
    
        #for th final training and testing accuracies and errors on the dataset
        probstest+= clf.predict_proba(X_test)*alphas[i]#multiply by aplhas?
        probstrain+= clf.predict_proba(X_train)*alphas[i]#multiply by aplhas?
    
    print (probstrain.shape, probstest.shape)
    return (probstrain, probstest)
    
    
    

In [17]:
# clf= DecisionTreeClassifier(max_depth= 2, max_leaf_nodes= 5)
# clf= clf.fit(X_train, Y_train)
# pred= clf.predict(X_test)
# print (np.unique(pred))
probstrain, probstest= boosting(X_train, Y_train, X_test, 10)

[7.14285714e-05 7.14285714e-05 7.14285714e-05 ... 7.14285714e-05
 7.14285714e-05 7.14285714e-05]
0 Iteration
[7.14285714e-05 1.42857143e-04 2.14285714e-04 ... 9.99857143e-01
 9.99928571e-01 1.00000000e+00]
0.8397857142857142
0.8283172761737663
1 Iteration
[8.20686590e-05 1.64137318e-04 2.46205977e-04 ... 9.99835863e-01
 9.99917931e-01 1.00000000e+00]
0.8484285714285714
0.8611644547009385
2 Iteration
[9.57669668e-05 1.91533934e-04 2.08642664e-04 ... 9.99887124e-01
 9.99982891e-01 1.00000000e+00]
0.8535714285714285
0.8814457426136013
3 Iteration
[1.14368222e-04 1.33987875e-04 1.54419713e-04 ... 9.99865200e-01
 9.99979568e-01 1.00000000e+00]
0.8655714285714285
0.9311784546798045
4 Iteration
[1.39782453e-04 1.63761870e-04 1.88733950e-04 ... 9.99835245e-01
 9.99975028e-01 1.00000000e+00]
0.8567142857142858
0.8941319176032873
5 Iteration
[1.73719737e-04 2.03521031e-04 2.34555993e-04 ... 9.99795245e-01
 9.99968965e-01 1.00000000e+00]
0.8816428571428572
1.004040181158683
6 Iteration
[2.8212145

In [18]:
accuracy= getAcc(Y_test, probstest)
accuracy2= getAcc(Y_train, probstrain)
print ("The accuracy on the train set for Boosting is: " + str(accuracy2))
print ("The accuracy on the test set for Boosting is: " + str(accuracy))

[ 0  2  4  5  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24 25]
[ 0  2  4  5  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24 25]
The accuracy on the train set for Boosting is: 0.5058571428571429
The accuracy on the test set for Boosting is: 0.5161666666666667


In [19]:
#5 fold cross validation for Boosting
randarray2= np.arange(len(X_train))
breakpt2= round(0.2*len(X_train))
print (breakpt2)
accuracies= np.zeros(5)
testaccuracies= np.zeros(5)
for i in range(5):
    xx_train= []
    yy_train= []
    xx_val= []
    yy_val= []
    for j in range(i*breakpt2, min((i+1)*breakpt2, len(X_train))):
        xx_val.append(X_train[randarray2[j]])
        yy_val.append(Y_train[randarray2[j]])
    for j in range(len(X_train)):
        if(j< i*breakpt2 or j>= (i+1)*breakpt2):
            xx_train.append(X_train[randarray2[j]])
            yy_train.append(Y_train[randarray2[j]])
    
    print (len(xx_val), len(xx_train))
    probs_train, probs_val= boosting(xx_train, yy_train, xx_val, 10)
    acc=  getAcc(yy_val, probs_val)
    accuracies[i]= acc
    print ("The accuracy on validation set for " + str(i) + " fold is: " + str(acc))
    probs_train, probs_test= boosting(xx_train, yy_train, X_test, 10)
    acc2=  getAcc(Y_test, probs_test)
    testaccuracies[i]= acc2
    #print ("The accuracy on validation set for " + str(i) + " fold is: " + str(acc))

print("The mean accuracy on Validation set is: " + str(np.mean(accuracies)))
print("The standard deviation of accuracies on Validation set is: " + str(np.std(accuracies)))
index= np.argmax(testaccuracies)
print("The best model during C.V. gives accuraacy of " + str(testaccuracies[index]) + " on the Test set")    
    


2800
2800 11200
[8.92857143e-05 8.92857143e-05 8.92857143e-05 ... 8.92857143e-05
 8.92857143e-05 8.92857143e-05]
0 Iteration
[8.92857143e-05 1.78571429e-04 2.67857143e-04 ... 9.99821429e-01
 9.99910714e-01 1.00000000e+00]
0.8385714285714285
0.8238185005578778
1 Iteration
[1.02668902e-04 2.05337803e-04 2.25102004e-04 ... 9.99794662e-01
 9.99897331e-01 1.00000000e+00]
0.8526785714285714
0.8778829332900747
2 Iteration
[1.19401185e-04 2.38802371e-04 2.61787607e-04 ... 9.99859969e-01
 9.99979370e-01 1.00000000e+00]
0.8502678571428571
0.8683517211854185
3 Iteration
[1.42991026e-04 2.85982051e-04 3.13508433e-04 ... 9.99832304e-01
 9.99975295e-01 1.00000000e+00]
0.8657142857142858
0.9317926018996555
4 Iteration
[2.71051144e-05 2.01846596e-04 2.35485076e-04 ... 9.99942704e-01
 9.99969809e-01 1.00000000e+00]
0.8525
0.8771725166888988
5 Iteration
[3.38363711e-05 2.51972976e-04 2.93965202e-04 ... 9.99928475e-01
 9.99962311e-01 1.00000000e+00]
0.8759821428571428
0.9774600707045233
6 Iteration
[5.90

0.83875
0.8244783663050048
1 Iteration
[1.02656713e-04 2.05313426e-04 3.07970139e-04 ... 9.99794687e-01
 9.99897343e-01 1.00000000e+00]
0.8491071428571428
0.8638076738111583
2 Iteration
[1.19738537e-04 1.41016994e-04 2.60755531e-04 ... 9.99858983e-01
 9.99978722e-01 1.00000000e+00]
0.8566964285714286
0.8940591864388421
3 Iteration
[1.42551555e-04 1.67884060e-04 1.91729318e-04 ... 9.99832116e-01
 9.99974667e-01 1.00000000e+00]
0.8752678571428572
0.9741806905023822
4 Iteration
[1.71907297e-04 2.02456543e-04 2.31212271e-04 ... 9.99797543e-01
 9.99969451e-01 1.00000000e+00]
0.886875
1.0296053229471083
5 Iteration
[2.12237601e-04 2.49953852e-04 2.85455817e-04 ... 9.99750046e-01
 9.99962284e-01 1.00000000e+00]
0.8609821428571428
0.9117356853381928
6 Iteration
[4.16776156e-05 8.75480240e-05 1.30725423e-04 ... 9.99912452e-01
 9.99954130e-01 1.00000000e+00]
0.8683035714285714
0.9430209498563632
7 Iteration
[4.93345480e-05 1.03632181e-04 1.11384063e-04 ... 9.99942430e-01
 9.99991765e-01 1.000000

In [20]:
#Bagging done right
def bagging(X_train, Y_train, X_test, N):
    uniq= np.unique(Y_train)
    probstrain= np.zeros([len(X_train), len(uniq)])
    probstest= np.zeros([len(X_test), len(uniq)])
    for i in range(N):
        print (str(i) + " Iteration")
        indices= []
        temp= np.zeros([len(X_train), len(X_train[0])])
        labeltemp= np.zeros(len(Y_train))
        for j in range(len(X_train)):
            a= random.randint(0, len(X_train)-1)#since both are inclusive otherwise
            indices.append(a)
            temp[j]= X_train[a]
            labeltemp[j]= Y_train[a]
        
        
        print(labeltemp.shape, temp.shape)
        clf= DecisionTreeClassifier(random_state= 0, max_depth= 2, max_leaf_nodes= 5)
        clf= clf.fit(temp, labeltemp)
        probstest+= clf.predict_proba(X_test)#multiply by alphas??
        probstrain+= clf.predict_proba(X_train) 
    
    return (probstrain, probstest)    
    

In [30]:
probstrain, probstest= bagging(X_train, Y_train, X_test, 10)

0 Iteration
(14000,) (14000, 16)
1 Iteration
(14000,) (14000, 16)
2 Iteration
(14000,) (14000, 16)
3 Iteration
(14000,) (14000, 16)
4 Iteration
(14000,) (14000, 16)
5 Iteration
(14000,) (14000, 16)
6 Iteration
(14000,) (14000, 16)
7 Iteration
(14000,) (14000, 16)
8 Iteration
(14000,) (14000, 16)
9 Iteration
(14000,) (14000, 16)


In [31]:
accuracy= getAcc(Y_test, probstest)
accuracy2= getAcc(Y_test, probstrain)
print ("The accuracy on the train set for Bagging is: " + str(accuracy2))
print ("The accuracy on the test set for Bagging is: " + str(accuracy))

[ 0  3  8 11 12 15 16 19 20]
[ 0  3  8 11 12 15 16 19 20]
The accuracy on the train set for Bagging is: 0.03716666666666667
The accuracy on the test set for Bagging is: 0.2555


In [35]:
#normalization on Bagging!
#MIN-MAX
temp= probstest
# print (temp.shape)
# for i in range(len(probstest)):
#     maxx= np.max(temp[i])
#     minn= np.min(temp[i])
#     temp[i]= (temp[i]-minn)/(maxx-minn)

#Z-score normalization
# for i in range(len(probstest)):
#     mean= np.mean(temp[i])
#     std= np.std(temp[i])
#     temp[i]= (temp[i]-mean)/std

#tanh normalization
for i in range(len(probstest)):
    temp[i]= np.tanh(temp[i])

accuracy= getAcc(Y_test, temp)
print ("The accuracy on the test set for Bagging is: " + str(accuracy))


[ 0  3  8 11 12 15 16 19 20]
The accuracy on the test set for Bagging is: 0.2555


In [34]:
a= np.array([1,2,3])
b= np.max(a)
a-= b
print (np.tanh(a))
print (a)

[-0.96402758 -0.76159416  0.        ]
[-2 -1  0]


In [ ]:
#5 fold cross validation for Bagging
randarray2= np.arange(len(X_train))
breakpt2= round(0.2*len(X_train))
print (breakpt2)
valaccuracies= np.zeros(5)
testaccuracies= np.zeros(5)
for i in range(5):
    xx_train= []
    yy_train= []
    xx_val= []
    yy_val= []
    for j in range(i*breakpt2, min((i+1)*breakpt2, len(X_train))):
        xx_val.append(X_train[randarray2[j]])
        yy_val.append(Y_train[randarray2[j]])
    for j in range(len(X_train)):
        if(j< i*breakpt2 or j>= (i+1)*breakpt2):
            xx_train.append(X_train[randarray2[j]])
            yy_train.append(Y_train[randarray2[j]])
    
    print (len(xx_val), len(xx_train))
    probs_train, probs_val= bagging(xx_train, yy_train, xx_val, 10)
    acc=  getAcc(yy_val, probs_val)
    valaccuracies[i]= acc
    print ("The accuracy on validation set for " + str(i) + " fold is: " + str(acc))
    probs_train, probs_test= bagging(xx_train, yy_train, X_test, 10)
    acc2=  getAcc(Y_test, probs_test)
    testaccuracies[i]= acc2
    #print ("The accuracy on validation set for " + str(i) + " fold is: " + str(acc))

print("The mean accuracy on Validation set is: " + str(np.mean(valaccuracies)))
print("The standard deviation of accuracies on Validation set is: " + str(np.std(valaccuracies)))
index= np.argmax(testaccuracies)
print("The best model during C.V. gives accuraacy of " + str(testaccuracies[index]) + " on the Test set")    
    
